# Web services with Flask
There are several web servers build with python:
A large web framework largely used in production code is: [Django](https://www.djangoproject.com/). For a more lightweight webserver with a friendlier learning curve we have Flask

Content is based on these ressources:  
https://blog.miguelgrinberg.com/post/designing-a-restful-api-with-python-and-flask   
https://blog.miguelgrinberg.com/post/restful-authentication-with-flask

For a super fast and simple webserver: [fast_api](https://fastapi.tiangolo.com/tutorial/)

## Flask in this docker container:

This is just for simple dev work. For production ready flask use the [flask docker project](https://github.com/HartmannDemoCode/flask_docker.git) or install flask directly on Digital Ocean as descriped below.

#### From inside the container: 
`docker exec -it notebookserver bash` run
`flask run --host=0.0.0.0` this will run flask with the env var: flask_app.py as specified in Dockerfile.  
To run another flask file put it in the shared volume (in the "flask" folder) and open the container like this with a new env var:  
`docker exec -it -e FLASK_APP="flask/flask_app2.py" notebookserver bash` and  
`flask run --host=0.0.0.0`

## Flask on local machine (not using docker)
If you have python installed with Anaconda on your pc, you got flask installed allready (check with `conda list` from inside your project folder).  
The following example code is the most basic implementation of a flask rest service:  
```python
#!flask/bin/python
from flask import Flask

app = Flask(__name__)

@app.route('/flask_app/')
def index():
    return "Hello, World from flask server!"

if __name__ == '__main__':
    app.run(debug=True)
```
Save it in a file: flask_app.py and run it from cli  
You can now access in browser from `http://127.0.0.1:5000/flask_app`

## Flask extended
Copy the file to flask_app_ext.py and add more http methods in the @app.route configuration

```python
#!flask/bin/python
from flask import Flask, jsonify, abort, request


app = Flask(__name__)

tasks = [
    {
        'id': 1,
        'title': 'Go shopping',
        'description': 'Milk, Cheese, Fruit, Wine', 
        'done': False
    },
    {
        'id': 2,
        'title': 'Study',
        'description': 'Learn about uWsgi server and flask with python', 
        'done': False
    }
]


@app.route('/todo/api/tasks', methods=['GET'])
def get_tasks():
    return jsonify({'tasks': tasks})


@app.route('/todo/api/tasks/<int:task_id>', methods=['GET'])
def get_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    return jsonify({'task': task[0]})

if __name__ == '__main__':
    app.run(debug=True)

@app.route('/todo/api/tasks', methods=['POST'])
def create_task():
    if not request.json or not 'title' in request.json:
        abort(400)
    task = {
        'id': tasks[-1]['id'] + 1,
        'title': request.json['title'],
        'description': request.json.get('description', ""),
        'done': False
    }
    tasks.append(task)
    return jsonify({'task': task}), 201

@app.route('/todo/api/tasks/<int:task_id>', methods=['PUT'])
def update_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    if not request.json:
        abort(400)
    if 'title' in request.json and type(request.json['title']) != unicode:
        abort(400)
    if 'description' in request.json and type(request.json['description']) is not unicode:
        abort(400)
    if 'done' in request.json and type(request.json['done']) is not bool:
        abort(400)
    task[0]['title'] = request.json.get('title', task[0]['title'])
    task[0]['description'] = request.json.get('description', task[0]['description'])
    task[0]['done'] = request.json.get('done', task[0]['done'])
    return jsonify({'task': task[0]})

@app.route('/todo/api/tasks/<int:task_id>', methods=['DELETE'])
def delete_task(task_id):
    task = [task for task in tasks if task['id'] == task_id]
    if len(task) == 0:
        abort(404)
    tasks.remove(task[0])
    return jsonify({'result': True})

```

## 01 Exercise (45 min)
1. Create a restful webservice that can respond to the following requests:  

|Method|URL|Example|Response|Description|
|--|--|--|--|--|
|GET|/datagenerator/api/person/{no}|/datagenerator/api/person/100|[{"id":1,"name":"Holger"},{},{}]|Returns 100 person objects|
|POST|datagenerator/api/person|POST json: {"name":"Henrietta"}|{"id":101,"name":"Henrietta"}|Adds a new Person object to the list of persons on the server|

2. Use a Mysql table to store and retrieve Person objects